# Python 2 HSUTCC: Session 7: Special Task

## Special Task (Bonus 8% - Deadline Thursday 27 Nov 2025)

Symbolic differentiation allows to calculate a value of a derivative for an expression in a form of a sequence of symbols. Humans are likely to make mistakes during a long monotonous work and calculating of a direvative even not that hard but tidious process. Systems of symbolic calculations, like in this <a href='https://www.wolframalpha.com/input?i=derivate+x%5E3+++(1+-+x)+/+(1+-+x%5E2)'>example</a> are dedicated to save humanity from the need to spend their time for a potentially wrong calcuations.

We will try to implement a simple system of symbolic differentiation on Python. The main type in our system will be a class `Expr` – an expression. All successor of the class should implement:

- `__call__` method which calculates a value of an expression in a specified context: the context bounds names of variables in the expression with specific values.
- `d` (Leibniz's notation of derivative) method which takes a name of a variable wrt (from “with respect to”) and returns an expression for a derivative with respect to this variable.

In [ ]:
class Expr:
    def __call__(self, **context):
        raise NotImplementedError()
    
    def d(self, wrt):
        raise NotImplementedError()
    
    def __str__(self):
        raise NotImplementedError()
    
    def __repr__(self):
        return str(self)

**Important**: For each class, implement a `__str__` method which should return a corresponding formula in a form of S-expression.

1. Implement class for two types of expressions: `Const` – a constant and `Var` – a variable. For convinience next we will use not a constructors of classes but their one-letter synonyms:

```python
V = Var
C = Const
```

Example of class usage:

---

```python
C(42)() # print(C(42)) -> 42
```

```terminal
42
```

---

```python
C(42).d(V("x"))() # print(C(42).d(V("x"))) -> 0
```

```terminal
0
```

---

```python
V("x")(x=42) # print(V("x")) -> x
```

```terminal
42
```

---

```python
V("x").d(V("x"))() # print(V("x").d(V("x"))) -> 1
```

```terminal
1
```

---

```python
V("x").d(V("y"))() # print(V("x").d(V("x"))) -> 0
```

---

In [ ]:
class Const(Expr):
    def __init__(self, value):
        self.value = value
    
    def __call__(self, **context):
        return self.value
    
    def d(self, wrt):
        # derivative of a constant = 0
        return Const(0)

    def __str__(self):
        return str(self.value)
    
class Var(Expr):
    def __init__(self, name):
        self.name = name
        
    def __call__(self, **context):
        return context[self.name]
    
    def d(self, wrt):
        # with respects to is an Expression, so extract its var name
        return Const(1) if self.name == wrt.name else Const(0)
    
    def __str__(self):
        return self.name
        

2. Implement classes for binary operations: `Sum`, `Product`, and `Fraction`. Binary operations by definition work with exactly two operands that’s why a constructor for all binary classes will be the same. It’s convinient to take it out into a separate base class:


In [ ]:
class BinOp(Expr):
    def __init__(self, expr1: Expr, expr2: Expr) -> None:
        self.expr1 = expr1
        self.expr2 = expr2

An example of using of some binary operations:

---

```python
# x + x^2
x = V("x")
Sum(x, Product(x, x)).d(x)(x=42) # (+ 1 (+ (* 1 x) (* x 1)))
```

```terminal
85
```

---

```python
Product(x, Sum(x, C(2)))(x=42) # (* x (+ x 2))
```

```terminal
1848
```

---

```python
Fraction(Product(x, V("y")), Sum(C(42), x)).d(x)(x=42, y=24)
```

```terminal
0.14285714285714285
```

---

```python
Fraction(Product(x, V("y")), Sum(C(42), x)).d(V("y"))(x=42, y=24)
```

```terminal
0.5
```

---

In [ ]:
class Sum(BinOp):
    def __call__(self, **context):
        return self.expr1(**context) + self.expr2(**context)
    
    def d(self, wrt):
        return Sum(self.expr1.d(wrt), self.expr2.d(wrt))
    
    def __str__(self):
        return f"(+ {self.expr1} {self.expr2})"

class Product(BinOp):
    def __call__(self, **context):
        return self.expr1(**context) * self.expr2(**context)
    
    def d(self, wrt):
        return Sum(
            Product(self.expr1.d(wrt), self.expr2),
            Product(self.expr1, self.expr2.d(wrt))
        )
    
    def __str__(self):
        return f"(* {self.expr1} {self.expr2})"

class Fraction(BinOp):
    def __call__(self, **context):
        return self.expr1(**context) / self.expr2(**context)
    
    def d(self, wrt):
        num = Sum(
            Product(self.expr1.d(wrt), self.expr2),
            Product(Const(-1), Product(self.expr1, self.expr2.d(wrt)))
        )
        den = Product(self.expr2, self.expr2)
        return Fraction(num, den)

    def __str__(self):
        return f"(/ {self.expr1} {self.expr2})"

3. Implement a class `Power` for an operation of an exponentiation operation. For simplicity you can expect that a power – is an expression consisting only of constants.

---

```python
Power(Fraction(V("x"), C(4)), C(2))(x=42) # (** (/ x 4) 2)
```

```terminal
110.25
```

---

```python
Power(Fraction(V("x"), C(4)), C(2)).d(V("x"))(x=42)
```

```terminal
5.25
```

---

In [ ]:
class Power(BinOp):
    def __call__(self, **context):
        return self.expr1(**context) ** self.expr2(**context)
    
    def d(self, wrt):
        """exponent must be a const"""
        if not isinstance(self.expr2, Const):
            raise ValueError("Exponent must be a constant expression.")
        
        constant = self.expr2.value
        
        # derivative: constant * (expr1 ** (constant - 1)) * expr1 )
        return Product(
            Const(constant),
            Product(Power(self.expr1, Const(constant - 1)),
                    self.expr1.d(wrt))
        )
        
    def __str__(self):
        return f"(** {self.expr1} {self.expr2})"

4. Implement an overloading of arithmetic operators for a base class Expr. Overall there are needed to be implement 7 operators: 2 unary and 5 binary:

```terminal
-e          e.__neg__()
+e          e.__pos__()
e1 + e2     e1.__add__(e2)
e1 - e2     e1.__sub__(e2)
e1 * e2     e1.__mul__(e2)
e1 / e2     e1.__truediv__(e2)
e1 ** e2    e1.__pow__(e2)
```

---

```python
((C(1) - V("x")) ** C(3) + V("x"))(x=12)
```

```terminal
-1319
```

---

In [ ]:
class Expr():
    def __call__(self, **context):
        raise NotImplementedError()

    def d(self, wrt):
        raise NotImplementedError()
    
    def __str__(self):
        raise NotImplementedError()
    
    def __repr__(self):
        raise NotImplementedError()

    # Overloading of arithmetic operators
    def __neg__(self):
        # -e = (* -1 e)
        return Product(Const(-1), self)
    
    def __pos__(self):
        # +e just returns the same expression
        return self
    
    # Binary operators
    def __add__(self, other):
        return Sum(self, other)
    
    def __sub__(self, other):
        return Sum(self, -other) # e1 - e2 = e1 + (-e2)
    
    def __mul__(self, other):
        return Product(self, other)
    
    def __truediv__(self, other):
        return Fraction(self, other)
    
    def __pow__(self, other):
        return Power(self, other)

In [ ]:
# To test your implementation, put all other classes here
# Restart the kernel, run the above and current cell, then do some testing

class Expr:
    def __call__(self, **context):
        """ evaluate the expression numerically (Every subclass will implement this.)"""
        raise NotImplementedError()
    
    def d(self, wrt):
        raise NotImplementedError()
    
    def __str__(self):
        raise NotImplementedError()

    def __repr__(self):
        return str(self)

    # ---------- Unary operators ----------

    def __neg__(self):
        # -e  -> (* -1 e)
        return Product(Const(-1), self)
    
    def __pos__(self):
        # +e just returns the same expression
        return self
    
    # ---------- Binary operators ----------

    def __add__(self, other):
        return Sum(self, other)
    
    def __sub__(self, other):
        # e1 - e2  =  e1 + (-e2)
        return Sum(self, -other)
    
    def __mul__(self, other):
        return Product(self, other)
    
    def __truediv__(self, other):
        return Fraction(self, other)
    
    def __pow__(self, other):
        return Power(self, other)


class Const(Expr): 
    def __init__(self, value):
        self.value = value
    
    def __call__(self, **context):
        return self.value
    
    def d(self, wrt):
        # derivative of a constant = 0
        return Const(0)

    def __str__(self):
        return str(self.value)


class Var(Expr):
    def __init__(self, name):
        self.name = name
        
    def __call__(self, **context):
        return context[self.name]
    
    def d(self, wrt):
        # wrt is an Expr, so extract its var name
        return Const(1) if self.name == wrt.name else Const(0)
    
    def __str__(self):
        return self.name 
    
    def __repr__(self):
        return str(self)


class BinOp(Expr): 
    def __init__(self, expr1: Expr, expr2: Expr) -> None:
        self.expr1 = expr1
        self.expr2 = expr2


class Sum(BinOp): 
    def __call__(self, **context):
        return self.expr1(**context) + self.expr2(**context)
    
    def d(self, wrt):
        return Sum(self.expr1.d(wrt), self.expr2.d(wrt))
    
    def __str__(self):
        return f"(+ {self.expr1} {self.expr2})"


class Product(BinOp):
    def __call__(self, **context):
        return self.expr1(**context) * self.expr2(**context)
    
    def d(self, wrt):
        return Sum(
            Product(self.expr1.d(wrt), self.expr2),
            Product(self.expr1, self.expr2.d(wrt))
        )
    
    def __str__(self):
        return f"(* {self.expr1} {self.expr2})"


class Fraction(BinOp):
    def __call__(self, **context):
        return self.expr1(**context) / self.expr2(**context)
    
    def d(self, wrt):
        num = Sum(
            Product(self.expr1.d(wrt), self.expr2),
            Product(Const(-1), Product(self.expr1, self.expr2.d(wrt)))
        )
        den = Product(self.expr2, self.expr2)
        return Fraction(num, den)

    def __str__(self):
        return f"(/ {self.expr1} {self.expr2})"


class Power(BinOp):
    def __call__(self, **context):
        return self.expr1(**context) ** self.expr2(**context)
    
    def d(self, wrt):
        """Exponent must be a Const."""
        if not isinstance(self.expr2, Const):
            raise ValueError("Exponent must be a constant expression.")
        
        c = self.expr2.value
        
        # derivative: c * (expr1 ** (c - 1)) * expr1'
        return Product(
            Const(c),
            Product(
                Power(self.expr1, Const(c - 1)),
                self.expr1.d(wrt)
            )
        )
        
    def __str__(self):
        return f"(** {self.expr1} {self.expr2})"
